In [ ]:
### 使用提示词完成任务

In [1]:
# 导入依赖库
import dashscope
import os

# 从环境变量中获取 API Key
dashscope.api_key = os.getenv('DASHSCOPE_API_KEY')

# 基于 prompt 生成文本
# 使用 deepseek-v3 模型
def get_completion(prompt, model="deepseek-v3"):
    messages = [{"role": "user", "content": prompt}]    # 将 prompt 作为用户输入
    response = dashscope.Generation.call(
        model=model,
        messages=messages,
        result_format='message',  # 将输出设置为message形式
        temperature=0,  # 模型输出的随机性，0 表示随机性最小
    )
    return response.output.choices[0].message.content  # 返回模型生成的文本
    
# 任务描述
instruction = """
你的任务是识别用户对手机流量套餐产品的选择条件。
每种流量套餐产品包含三个属性：名称，月费价格，月流量。
根据用户输入，识别用户在上述三种属性上的需求是什么。
"""

# 用户输入
input_text = """
办个100G的套餐。
"""

# prompt 模版。instruction 和 input_text 会被替换为上面的内容
prompt = f"""
# 目标
{instruction}

# 用户输入
{input_text}
"""

print("==== Prompt ====")
print(prompt)
print("================")

# 调用大模型
response = get_completion(prompt)
print(response)

==== Prompt ====

# 目标

你的任务是识别用户对手机流量套餐产品的选择条件。
每种流量套餐产品包含三个属性：名称，月费价格，月流量。
根据用户输入，识别用户在上述三种属性上的需求是什么。


# 用户输入

办个100G的套餐。


# 分析结果

根据用户输入"办个100G的套餐"，可以识别出以下选择条件：

1. 月流量需求：用户明确要求100GB的月流量
   - 识别依据：直接提到"100G"

2. 其他属性：
   - 名称：未提及具体偏好
   - 月费价格：未提及具体金额要求

因此，用户的核心需求是选择一个月流量为100GB的手机套餐产品，对套餐名称和价格没有明确限制。


### JSON格式

In [2]:
# 输出格式
output_format = """
以 JSON 格式输出
"""

# 稍微调整下咒语，加入输出格式
prompt = f"""
# 目标
{instruction}

# 输出格式
{output_format}

# 用户输入
{input_text}
"""

# 调用大模型，指定用 JSON mode 输出
response = get_completion(prompt)
print(response)

```json
{
  "名称": null,
  "月费价格": null,
  "月流量": "100G"
}
```


### CoT示例

In [4]:
instruction = """
给定一段用户与手机流量套餐客服的对话，。
你的任务是判断客服的回答是否符合下面的规范：

- 必须有礼貌
- 必须用官方口吻，不能使用网络用语
- 介绍套餐时，必须准确提及产品名称、月费价格和月流量总量。上述信息缺失一项或多项，或信息与事实不符，都算信息不准确
- 不可以是话题终结者

已知产品包括：

经济套餐：月费50元，月流量10G
畅游套餐：月费180元，月流量100G
无限套餐：月费300元，月流量1000G
校园套餐：月费150元，月流量200G，限在校学生办理
"""

# 输出描述
output_format = """
如果符合规范，输出：Y
如果不符合规范，输出：N
"""

context = """
用户：你们有什么流量大的套餐
客服：亲，我们现在正在推广无限套餐，每月300元就可以享受1000G流量，您感兴趣吗？
"""

# cot = ""
cot = "请一步一步分析对话"

prompt = f"""
# 目标
{instruction}
{cot}

# 输出格式
{output_format}

# 对话上下文
{context}
"""

response = get_completion(prompt)
print(response)

让我们一步一步分析客服的回答是否符合规范：

1. 礼貌性：
   - 使用了"亲"这样的网络用语，不符合"必须用官方口吻"的要求
   - 整体语气是礼貌的，但用词不规范

2. 官方口吻：
   - 使用了"亲"这样的非正式网络用语，不符合要求

3. 信息准确性：
   - 准确提到了产品名称("无限套餐")
   - 准确提到了月费价格("300元")
   - 准确提到了月流量总量("1000G")
   - 信息完整且准确

4. 话题延续性：
   - 以询问"您感兴趣吗？"结尾，不是话题终结者

主要问题在于使用了非官方的网络用语"亲"，因此不符合规范。

最终判断：N


### 使用Prompt调优Prompt

In [5]:
user_prompt = """
做一个手机流量套餐的客服代表，叫小瓜。可以帮助用户选择最合适的流量套餐产品。可以选择的套餐包括：
经济套餐，月费50元，10G流量；
畅游套餐，月费180元，100G流量；
无限套餐，月费300元，1000G流量；
校园套餐，月费150元，200G流量，仅限在校生。"""

instruction = """
你是一名专业的提示词创作者。你的目标是帮助我根据需求打造更好的提示词。

你将生成以下部分：
提示词：{根据我的需求提供更好的提示词}
优化建议：{用简练段落分析如何改进提示词，需给出严格批判性建议}
问题示例：{提出最多3个问题，以用于和用户更好的交流}
"""

prompt = f"""
# 目标
{instruction}

# 用户提示词
{user_prompt}
"""

response = get_completion(prompt)
print(response)

# 优化后的提示词

作为手机流量套餐客服代表"小瓜"，你需要：
1. 热情友好地介绍自己，明确服务范围
2. 主动询问用户需求(如每月流量使用量、预算范围、是否在校生等)
3. 根据用户需求推荐最合适的套餐，清晰说明各套餐特点：
   - 经济套餐(50元/10G)：适合轻度用户
   - 畅游套餐(180元/100G)：适合中度用户
   - 无限套餐(300元/1000G)：适合重度用户
   - 校园套餐(150元/200G)：仅限在校生
4. 提供专业比较建议，考虑性价比
5. 确认用户选择前核实资格条件(如校园套餐需验证学生身份)
6. 保持耐心，用简单语言解释专业术语

# 优化建议

1. 角色定位更清晰：明确了客服代表的专业形象和服务标准
2. 交互流程更完整：从自我介绍到需求分析再到推荐确认，形成闭环
3. 信息呈现更系统：套餐对比采用结构化展示，便于用户理解
4. 增加了资格验证环节：避免后续纠纷，特别是校园套餐
5. 补充了服务态度要求：确保用户体验友好

建议增加：
- 常见问题应对策略(如超出流量怎么办)
- 优惠活动介绍(如新用户优惠)
- 转套餐流程说明
- 用户满意度确认环节

# 问题示例

1. 您平均每月使用多少流量？这样我可以更精准地推荐套餐
2. 您的预算范围大概是多少？我们可以根据预算找到性价比最高的选择
3. 您目前是在校学生吗？这关系到是否可以办理特别优惠的校园套餐
